# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
``` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
```
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>
6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

In [102]:
import pandas as pd
import requests

In [103]:
# 1) seu código aqui
df = pd.read_csv('SINASC_RO_2019.csv')
print(f'Número total de registros: {len(df)}')
print(f'Há linhas duplicadas: {len(df) != len(df.drop_duplicates())}')

Número total de registros: 27028
Há linhas duplicadas: False


In [104]:
# 2) seu código aqui
print(f'Valores missing por variável:\n{df.isnull().sum()}')

Valores missing por variável:
ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64


In [105]:
# 3) seu código aqui
df_interesse = pd.read_csv('SINASC_RO_2019.csv')[[
      'LOCNASC', 
      'IDADEMAE', 
      'ESTCIVMAE', 
      'ESCMAE', 
      'QTDFILVIVO', 
      'GESTACAO', 
      'GRAVIDEZ', 
      'CONSULTAS', 
      'APGAR5']]
print(f'DataFrame com colunas de interesse:\n{df_interesse}')

DataFrame com colunas de interesse:
       LOCNASC  IDADEMAE  ESTCIVMAE           ESCMAE  QTDFILVIVO  \
0            1        19        5.0      8 a 11 anos         0.0   
1            1        29        2.0      8 a 11 anos         1.0   
2            1        37        9.0      8 a 11 anos         2.0   
3            1        30        5.0  12 anos ou mais         0.0   
4            1        30        2.0      8 a 11 anos         1.0   
...        ...       ...        ...              ...         ...   
27023        1        32        2.0  12 anos ou mais         1.0   
27024        1        19        1.0      8 a 11 anos         0.0   
27025        1        24        2.0      8 a 11 anos         0.0   
27026        1        21        2.0      8 a 11 anos         1.0   
27027        1        18        5.0      8 a 11 anos         1.0   

              GESTACAO GRAVIDEZ  CONSULTAS  APGAR5  
0      37 a 41 semanas    Única          4    10.0  
1      37 a 41 semanas    Única          

In [106]:
# 3.1) seu código aqui
print(f'Valores missing nas colunas de interesse:\n{df_interesse.isnull().sum()}')

Valores missing nas colunas de interesse:
LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64


In [107]:
# 4) seu código aqui
df_sem_apgar5_missing = df_interesse.dropna(subset=['APGAR5'])
print(f'\nDataFrame após remover registros com Apgar5 não preenchido:\n{df_sem_apgar5_missing}')
num_registros_sem_apgar5_missing = len(df_sem_apgar5_missing)
missings_apgar5 = df_sem_apgar5_missing.isnull().sum()
print(f'Número de registros após remoção: {num_registros_sem_apgar5_missing}')
print(f'Valores missing nas colunas de interesse após remoção:\n{missings_apgar5}')


DataFrame após remover registros com Apgar5 não preenchido:
       LOCNASC  IDADEMAE  ESTCIVMAE           ESCMAE  QTDFILVIVO  \
0            1        19        5.0      8 a 11 anos         0.0   
1            1        29        2.0      8 a 11 anos         1.0   
2            1        37        9.0      8 a 11 anos         2.0   
3            1        30        5.0  12 anos ou mais         0.0   
4            1        30        2.0      8 a 11 anos         1.0   
...        ...       ...        ...              ...         ...   
27023        1        32        2.0  12 anos ou mais         1.0   
27024        1        19        1.0      8 a 11 anos         0.0   
27025        1        24        2.0      8 a 11 anos         0.0   
27026        1        21        2.0      8 a 11 anos         1.0   
27027        1        18        5.0      8 a 11 anos         1.0   

              GESTACAO GRAVIDEZ  CONSULTAS  APGAR5  
0      37 a 41 semanas    Única          4    10.0  
1      37 a 41 s

In [108]:
# 5) seu código aqui
df_interesse = pd.read_csv('SINASC_RO_2019.csv')[['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']]
print(f'DataFrame com colunas de interesse:\n{df_interesse}\nValores missing nas colunas de interesse:\n{df_interesse.isnull().sum()}')
df_interesse.replace({'ESTCIVMAE': 9, 'CONSULTAS': 9}, pd.NA, inplace=True)
print(f'\nDataFrame após substituir código 9 por NaN em ESTCIVMAE e CONSULTAS:\n{df_interesse}')

DataFrame com colunas de interesse:
       LOCNASC  IDADEMAE  ESTCIVMAE           ESCMAE  QTDFILVIVO  \
0            1        19        5.0      8 a 11 anos         0.0   
1            1        29        2.0      8 a 11 anos         1.0   
2            1        37        9.0      8 a 11 anos         2.0   
3            1        30        5.0  12 anos ou mais         0.0   
4            1        30        2.0      8 a 11 anos         1.0   
...        ...       ...        ...              ...         ...   
27023        1        32        2.0  12 anos ou mais         1.0   
27024        1        19        1.0      8 a 11 anos         0.0   
27025        1        24        2.0      8 a 11 anos         0.0   
27026        1        21        2.0      8 a 11 anos         1.0   
27027        1        18        5.0      8 a 11 anos         1.0   

              GESTACAO GRAVIDEZ  CONSULTAS  APGAR5  
0      37 a 41 semanas    Única          4    10.0  
1      37 a 41 semanas    Única          

In [109]:
# 6) seu código aqui
df_sem_apgar5_missing['QTDFILVIVO'].fillna(0, inplace=True)
print(f'\nDataFrame após substituir valores faltantes em QTDFILVIVO por zero:\n{df_sem_apgar5_missing}')


DataFrame após substituir valores faltantes em QTDFILVIVO por zero:
       LOCNASC  IDADEMAE  ESTCIVMAE           ESCMAE  QTDFILVIVO  \
0            1        19        5.0      8 a 11 anos         0.0   
1            1        29        2.0      8 a 11 anos         1.0   
2            1        37        9.0      8 a 11 anos         2.0   
3            1        30        5.0  12 anos ou mais         0.0   
4            1        30        2.0      8 a 11 anos         1.0   
...        ...       ...        ...              ...         ...   
27023        1        32        2.0  12 anos ou mais         1.0   
27024        1        19        1.0      8 a 11 anos         0.0   
27025        1        24        2.0      8 a 11 anos         0.0   
27026        1        21        2.0      8 a 11 anos         1.0   
27027        1        18        5.0      8 a 11 anos         1.0   

              GESTACAO GRAVIDEZ  CONSULTAS  APGAR5  
0      37 a 41 semanas    Única          4    10.0  
1      3

In [110]:
# 7) seu código aqui
df_sem_apgar5_missing['IDADEMAE'].fillna(df_sem_apgar5_missing['IDADEMAE'].median(), inplace=True)
print(f'\nDataFrame após substituir valores faltantes em IDADEMAE pela mediana:\n{df_sem_apgar5_missing.head()}')
df_sem_apgar5_missing['ESCMAE'].fillna(df_sem_apgar5_missing['ESCMAE'].mode()[0], inplace=True)
print(f'\nDataFrame após substituir valores faltantes em ESCMAE pela categoria mais comum:\n{df_sem_apgar5_missing.head()}')
df_sem_apgar5_missing[['GESTACAO', 'GRAVIDEZ', 'LOCNASC']] = df_sem_apgar5_missing[['GESTACAO', 'GRAVIDEZ', 'LOCNASC']].fillna(df_sem_apgar5_missing.mode().iloc[0])
print(f'\nDataFrame após substituir valores faltantes em variáveis categóricas:\n{df_sem_apgar5_missing.head()}')
df_sem_apgar5_missing['CONSULTAS'].fillna(df_sem_apgar5_missing['CONSULTAS'].median(), inplace=True)
print(f'\nDataFrame após substituir valores faltantes em CONSULTAS pela mediana:\n{df_sem_apgar5_missing.head()}')


DataFrame após substituir valores faltantes em IDADEMAE pela mediana:
   LOCNASC  IDADEMAE  ESTCIVMAE           ESCMAE  QTDFILVIVO         GESTACAO  \
0        1        19        5.0      8 a 11 anos         0.0  37 a 41 semanas   
1        1        29        2.0      8 a 11 anos         1.0  37 a 41 semanas   
2        1        37        9.0      8 a 11 anos         2.0  37 a 41 semanas   
3        1        30        5.0  12 anos ou mais         0.0  37 a 41 semanas   
4        1        30        2.0      8 a 11 anos         1.0  37 a 41 semanas   

  GRAVIDEZ  CONSULTAS  APGAR5  
0    Única          4    10.0  
1    Única          4     9.0  
2    Única          4    10.0  
3    Única          3    10.0  
4    Única          4    10.0  

DataFrame após substituir valores faltantes em ESCMAE pela categoria mais comum:
   LOCNASC  IDADEMAE  ESTCIVMAE           ESCMAE  QTDFILVIVO         GESTACAO  \
0        1        19        5.0      8 a 11 anos         0.0  37 a 41 semanas   
1     

# Justificativa das tratativas ex7:

IDADEMAE: Optei por preencher com a mediana, que é o valor que está no meio dos dados, para manter as idades próximas do que já tínhamos.

ESCMAE: Preenchi com a categoria que aparecia mais vezes, a mais comum, para não mudar muito a distribuição das informações.

GESTACAO, GRAVIDEZ, LOCNASC: Coloquei os valores mais comuns, acreditando que representam bem a maioria dos casos.

CONSULTAS: Optei por usar a mediana para não ser muito influenciado por valores muito altos ou baixos.

Busquei equilibrar entre manter as características originais dos dados e inserir informações que fazem sentido

In [111]:
# 8) seu código aqui
df_sem_apgar5_missing['APGAR_CATEGORIA'] = pd.cut(df_sem_apgar5_missing['APGAR5'], bins=[-1, 3, 5, 7, 10], labels=['Severe Asphyxia', 'Moderate Asphyxia', 'Mild Asphyxia', 'Normal'])
print(f'\nDataFrame após categorizar Apgar:\n{df_sem_apgar5_missing}')
frequencias_apgar_categoria = df_sem_apgar5_missing['APGAR_CATEGORIA'].value_counts()
print(f'Frequências da categorização de Apgar:\n{frequencias_apgar_categoria}')


DataFrame após categorizar Apgar:
       LOCNASC  IDADEMAE  ESTCIVMAE           ESCMAE  QTDFILVIVO  \
0            1        19        5.0      8 a 11 anos         0.0   
1            1        29        2.0      8 a 11 anos         1.0   
2            1        37        9.0      8 a 11 anos         2.0   
3            1        30        5.0  12 anos ou mais         0.0   
4            1        30        2.0      8 a 11 anos         1.0   
...        ...       ...        ...              ...         ...   
27023        1        32        2.0  12 anos ou mais         1.0   
27024        1        19        1.0      8 a 11 anos         0.0   
27025        1        24        2.0      8 a 11 anos         0.0   
27026        1        21        2.0      8 a 11 anos         1.0   
27027        1        18        5.0      8 a 11 anos         1.0   

              GESTACAO GRAVIDEZ  CONSULTAS  APGAR5 APGAR_CATEGORIA  
0      37 a 41 semanas    Única          4    10.0          Normal  
1      37 

In [112]:
# 9) seu código aqui
df_sem_apgar5_missing.columns = df_sem_apgar5_missing.columns.str.lower().str.replace(' ', '_')
print(f'\nDataFrame após renomear variáveis para snake case:\n{df_sem_apgar5_missing}')


DataFrame após renomear variáveis para snake case:
       locnasc  idademae  estcivmae           escmae  qtdfilvivo  \
0            1        19        5.0      8 a 11 anos         0.0   
1            1        29        2.0      8 a 11 anos         1.0   
2            1        37        9.0      8 a 11 anos         2.0   
3            1        30        5.0  12 anos ou mais         0.0   
4            1        30        2.0      8 a 11 anos         1.0   
...        ...       ...        ...              ...         ...   
27023        1        32        2.0  12 anos ou mais         1.0   
27024        1        19        1.0      8 a 11 anos         0.0   
27025        1        24        2.0      8 a 11 anos         0.0   
27026        1        21        2.0      8 a 11 anos         1.0   
27027        1        18        5.0      8 a 11 anos         1.0   

              gestacao gravidez  consultas  apgar5 apgar_categoria  
0      37 a 41 semanas    Única          4    10.0          No